In [7]:
import numpy as np

In [8]:
import pandas as pd
df_edges = pd.read_csv('edges_h1p0_less_2_20211121.csv')
edges = df_edges.values.tolist()

In [27]:
len(edges)

102

In [15]:
import json
with open('VG-SGG-dicts.json', 'r') as fin:
    scene_graph_meta = json.load(fin)

In [10]:
from utils import load_obj
label2id = load_obj('label2id_150_zareian_code_20211017')
pred2id = load_obj('pred2id_old')
id2label = {}
for l, i in label2id.items():
    if i not in id2label:
        id2label[i] = []
    id2label[i].append(l) 
id2pred = {}
for l, i in pred2id.items():
    if i not in id2pred:
        id2pred[i] = []
    id2pred[i].append(l) 


In [11]:
import numpy as np
edges_ent2ent = np.zeros((5, 151, 151))
edges_pred2pred = np.zeros((3, 51, 51))
edges_ent2pred = np.zeros((1, 151, 51))
edges_pred2ent = np.zeros((1, 51, 151))
# edges_ent2ent = np.zeros((5, len(label2synset_combined), len(label2synset_combined)))
# edges_pred2pred = np.zeros((3, len(preds_old), len(preds_old)))
# edges_ent2pred = np.zeros((1, len(label2synset_combined), len(preds_old)))
# edges_pred2ent = np.zeros((1, len(preds_old), len(label2synset_combined)))
print(edges_ent2ent.shape)
print(edges_pred2pred.shape)
print(edges_ent2pred.shape)
print(edges_pred2ent.shape)

(5, 151, 151)
(3, 51, 51)
(1, 151, 51)
(1, 51, 151)


In [18]:
for edge in edges:
    if edge[2] != '/r/IsA':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        raise
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[1, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

In [19]:
for edge in edges:
    if edge[2] != '/r/PartOf':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        raise
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[2, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[3, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1


In [20]:
for edge in edges:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        if edge[0] not in id2pred or edge[1] not in id2pred:
            print(edge)
            raise
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1


In [21]:
for edge in edges:
    if edge[2] != '/r/MannerOf':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        raise
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[0, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[1, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1
        

In [23]:
for edge in edges:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        if edge[0] not in id2label or edge[1] not in id2label:
            raise
        continue
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1


In [24]:
for edge in edges:
    if edge[2] != '/r/UsedFor':
        continue
    if edge[0] not in id2label or edge[1] not in id2pred:
        print(edge)
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_ent2pred[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2ent[0, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

In [25]:
print(edges_ent2ent.sum(axis=(1,2)))
print(edges_pred2pred.sum(axis=(1,2)))
print(edges_ent2pred.sum(axis=(1,2)))
print(edges_pred2ent.sum(axis=(1,2)))


[30. 30. 37. 37. 14.]
[13. 13. 10.]
[16.]
[16.]


In [32]:
# edge_dict_conceptnet = {
#     'conceptnet_ent2ent': edges_ent2ent,
#     'conceptnet_pred2pred': edges_pred2pred,
#     'conceptnet_ent2pred': edges_ent2pred,
#     'conceptnet_pred2ent': edges_pred2ent,
# }
edge_dict_conceptnet = {
    'edges_ent2ent': edges_ent2ent,
    'edges_pred2pred': edges_pred2pred,
    'edges_ent2pred': edges_ent2pred,
    'edges_pred2ent': edges_pred2ent,
}

In [33]:
import os, pickle
with open(os.path.join('conceptnet_edges_h1p0_less_2_102_20211017.pkl'), 'wb') as fout:
    pickle.dump(edge_dict_conceptnet, fout)

In [21]:
# with open(os.path.join('conceptnet_edges_h1p0_less_2_20211017.pkl'), 'rb') as fout:
#     edge_dict_conceptnet = pickle.load(fout)

In [22]:
edges_ent2ent = edge_dict_conceptnet['conceptnet_ent2ent']
edges_pred2pred = edge_dict_conceptnet['conceptnet_pred2pred']
edges_ent2pred = edge_dict_conceptnet['conceptnet_ent2pred']
edges_pred2ent = edge_dict_conceptnet['conceptnet_pred2ent']

In [23]:
print(edges_ent2ent.sum(axis=(1,2)))
print(edges_pred2pred.sum(axis=(1,2)))
print(edges_ent2pred.sum(axis=(1,2)))
print(edges_pred2ent.sum(axis=(1,2)))

[30. 30. 37. 37. 14.]
[13. 13. 14.]
[16.]
[16.]
